In [1]:
import sys
# a) SchemasToGraph
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import os
import time
import ast
# b.1) SignatureEncoding
from sentence_transformers import SentenceTransformer, util
# b.2) LinkabilityAssessor
from sklearn.metrics import mean_squared_error
import sklearn
import sklearn.decomposition
# c) LinkabilityCorrelator
# d) SemanticMatcher
from itertools import product

In [2]:

class Entity: #could be table or column
    def __init__(
        self, id, name, type, schema, parent_id, datatype=None, constraints=None, text_sequence=None
    ):
        self.id = id
        self.name = name
        self.type = type
        self.schema = schema
        self.parent_id = parent_id
        self.datatype = datatype
        self.constraints = constraints
        self.text_sequence = text_sequence
        
    def __repr__(self):
        return (f"Entity({self.id}, {self.datatype}, {self.schema}, {self.parent_id}")
    def get_text_sequence(self):
        return self.text_sequence
    def get_instance_sequence(self):
        return self.instance_sequence

In [3]:
def encode_signatures_from_df(df_graph, model_name='sentence-transformers/all-mpnet-base-v2', serialization="text_sequence", instance_serialization="instance_sequence"):
    entities = []
    model = SentenceTransformer(model_name)

    df_graph = df_graph[df_graph.type != "schema"].reset_index(drop=True)
        
    st = time.time()
    df_graph[serialization] = df_graph[serialization].astype(str) 
    text_sequence = model.encode(df_graph[serialization].values)

    if instance_serialization:
        df_graph[instance_serialization] = df_graph[instance_serialization].astype(str) 
        instance_sequence = model.encode(df_graph[instance_serialization].values)
        text_sequence = np.concatenate((text_sequence, instance_sequence), axis=1)
    et = time.time()
    
    print("Elapsed time for encoding signatures:" + str(et - st))

    for index, row in df_graph.reset_index(drop=True).iterrows():
        entity = Entity(
            id=row["id"],
            name= row["name"],
            type=row["type"],
            datatype=row["datatype"],
            schema=row["schema"],
            parent_id=row["parent_id"],
            text_sequence=text_sequence[index]
            )
        entities.append(entity)

    return entities;

In [9]:
df_graph = pd.read_csv("C:/Users/leona/Desktop/schemas/schema_graph.csv")
print("Schema Graph Metadata:")
print("# Schemas: "+ str(len(df_graph[df_graph.type=="schema"])))
print("# Tables: "+ str(len(df_graph[df_graph.type=="table"])))
print("# Attributes: "+ str(len(df_graph[df_graph.type=="attribute"])))
    


Schema Graph Metadata:
# Schemas: 4
# Tables: 34
# Attributes: 253


In [11]:

df_graph['text_sequence'] = df_graph['text_sequence'].astype(str)
entities = encode_signatures_from_df(df_graph, serialization="text_sequence", instance_serialization=None)

Elapsed time for encoding signatures:6.539335012435913
